# Botón Para Esconder Código

In [12]:
#Sacado de https://mljar.com/blog/jupyter-notebook-hide-code/
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Aprete aqui para visualizar/esconder bloques de codigo."></form>''')

## Libraries y Base

In [13]:
import pandas as pd
import numpy as np
import os

# Parte I: Análisis de la base de hogares y cálculo de pobreza

1) Consideramos que hay varias variables que podrían resultar importantes a la hora de mejorar el ejercicio del TP2. A continuación, divididas por sección:

Identificación:
- REGION: La pobreza no es homogenea por regiones, asumimos que según sea la región de la vivienda, la probabilidad de ser pobre varía. 

Características de la Vivienda:
- IV5/IV6/IV7/IV12_3: Ciertas caracteristicas de la vivienda que pueden demarcar pobreza. Relacionadas con necesidades básicas. Ejemplo agua o tipo de techo. IV12_3 si es o no villa de emergencia.

Características Habitacionales del Hogar:
- II7:Régimen de tenencia, según cual sea, relacionado con si es pobre o no. Ejemplo, si es ocupante de hecho, asumo que es más probable que sea pobre.
- II8: Combustible, relacionado con necesidades básicas. 

Estrategias del Hogar:
- V5: Subsidios y ayuda social por lo general va destinado a gente que lo necesita, entre las que se encuentran personas de bajos recursos/pobres.
- V17/V19_A/V12/V4: Son variables relacionadas a la necesidad de conseguir un ingreso extra. Ejemplo, menores de edad trabajando o vender pertenencias.  Comunican necesidad, relacionadas con pobreza.

Resumen del Hogar: 
- IX_MEN10: Mayor cantidad de hijos en hogares de menos recursos. 

Recordamos que esto es adicional a lo presentado en el punto 6 de la parte 2 del TP2, en el cual seleccionamos las siguientes variables de la base individual: CH04, CH06, CH08, CH09, CH10, CH11, CH12, CH13, NIVEL_ED, ESTADO, PP02E, PP02H, PP02I.

2) 

In [18]:
#Abrimos la base
hogar = pd.read_excel('./EPH_usu_1er_Trim_2022_xlsx/usu_hogar_T122.xlsx')

#Eliminamos todos los aglomerados que no correspondan a CABA (cód 32) o GBA (cód 33)
hogar_cortado = hogar.query("AGLOMERADO in (32, 33)") # la manera mas rapida y facil de hacerlo

print(len(hogar_cortado))

hogar_cortado.sample(10)

2392


CODUSU  ANO4  TRIMESTRE  NRO_HOGAR  REALIZADA  \
7992   TQRMNOPWXHKLKUCDEIIAD00779923  2022          1          1          1   
7145   TQRMNOPTQHJMTQCDEIJAH00693075  2022          1          1          1   
4170   TQRMNOPQUHLKMOCDEIIAD00779771  2022          1          1          1   
12732  TQRMNOPRTHMMOOCDEIIAD00780020  2022          1          1          1   
2321   TQRMNOQQWHLOLRCDEIJAH00719572  2022          1          2          1   
3158   TQRMNOPVUHLOLSCDEIJAH00719038  2022          1          1          1   
3698   TQRMNORUWHKKSUCDEIJAH00780164  2022          1          1          1   
12529  TQRMNOQXWHKONPCDEIJAH00780690  2022          1          1          1   
13684  TQRMNOQTSHLLKPCDEIIAD00718527  2022          1          1          1   
12433  TQRMNORUSHJMTLCDEIJAH00719373  2022          1          1          1   

       REGION MAS_500  AGLOMERADO  PONDERA  IV1  ... GDECCFR  PDECCFR  \
7992        1       S          32     1102    1  ...    12.0      NaN   
7145        1       S          33     1314    1  ...     9.0      NaN   
4170        1       S          32      705    1  ...     7.0      NaN   
12732       1       S          32     1308    4  ...     7.0      NaN   
2321        1       S          33     3978    1  ...     4.0      NaN   
3158        1       S          33     3434    1  ...     6.0      NaN   
3698        1       S          33     2870    1  ...    12.0      NaN   
12529       1       S          33     1206    1  ...    12.0      NaN   
13684       1       S          32     1499    2  ...    12.0      NaN   
12433       1       S          33     1309    2  ...     4.0      NaN   

       ADECCFR PONDIH  VII1_1  VII1_2  VII2_1  VII2_2 VII2_3  VII2_4  
7992        12      0      96       0       1       0      0       0  
7145        10   2419       1       2      98       0      0       0  
4170         4    974       1       0       2       0      0       0  
12732        5   1453       1       0      98       0      0       0  
2321         4   3262       1       0      98       0      0       0  
3158         7   6938       1       0       3       0      0       0  
3698        12      0       2       0      96       0      0       0  
12529       12      0       2       0      98       0      0       0  
13684       12      0       1       0       2       0      0       0  
12433        5   2306       2       0       1       3      4       0  

[10 rows x 88 columns]

3) 

In [19]:
#Importamos tabla individual 
#Abrimos la base
individual = pd.read_excel('./EPH_usu_1er_Trim_2022_xlsx/usu_individual_T122.xlsx')

#Eliminamos todos los aglomerados que no correspondan a CABA (cód 32) o GBA (cód 33)
individual_cortado = individual.query("AGLOMERADO in (32, 33)") # la manera mas rapida y facil de hacerlo

print(len(individual_cortado))

individual_cortado.sample(10)



6706


,CODUSU,ANO4,TRIMESTRE,NRO_HOGAR,COMPONENTE,H15,REGION,MAS_500,AGLOMERADO,PONDERA,...,PDECIFR,ADECIFR,IPCF,DECCFR,IDECCFR,RDECCFR,GDECCFR,PDECCFR,ADECCFR,PONDIH
12455,TQRMNOQVUHJLRUCDEIJAH00780403,2022,1,1,3,1,1,S,33,4505,...,NaN,12,0.00,12,NaN,12,12.0,NaN,12,0
39025,TQRMNORUUHKOSOCDEIJAH00780775,2022,1,1,1,1,1,S,33,5287,...,NaN,12,0.00,12,NaN,12,12.0,NaN,12,0
5098,TQRMNOPXSHMKMTCDEIIAD00703139,2022,1,1,1,1,1,S,32,1349,...,NaN,1,29000.00,4,NaN,4,4.0,NaN,2,1773
38923,TQTMNOPVRHLLROCDEIJAH00780828,2022,1,1,1,1,1,S,33,1908,...,NaN,10,145000.00,10,NaN,10,10.0,NaN,10,4120
18865,TQRMNOQQTHKNRUCDEIJAH00780536,2022,1,1,1,1,1,S,33,692,...,NaN,1,7500.00,1,NaN,1,1.0,NaN,1,1181
45733,TQRMNOQVPHLNKSCDEIJAH00719529,2022,1,1,2,1,1,S,33,3255,...,NaN,12,0.00,0,NaN,0,0.0,NaN,0,3167
16914,TQRMNOQRQHJOLLCDEIJAH00699552,2022,1,1,3,0,1,S,33,5934,...,NaN,12,0.00,12,NaN,12,12.0,NaN,12,0
35653,TQRMNOSWRHJMMPCDEIJAH00693030,2022,1,1,2,1,1,S,33,759,...,NaN,8,50666.67,7,NaN,7,7.0,NaN,8,1584
34843,TQRMNOQWVHKKSNCDEIJAH00780739,2022,1,1,2,1,1,S,33,1930,...,NaN,10,238000.00,10,NaN,10,10.0,NaN,10,4562
7114,TQRMNOSXUHJMTRCDEIJAH00693085,2022,1,1,5,1,1,S,33,1517,...,NaN,12,0.00,12,NaN,12,12.0,NaN,12,0


In [20]:
#Mergeamos ambos DF
eph = individual_cortado.merge(hogar_cortado,how='left',on=['CODUSU', 'NRO_HOGAR'], suffixes=('', '_y'))

#tipos y nombres
print(eph.dtypes)

#notamos que hay columnas repetidas
eph['ITF'].equals(eph['ITF_y'])

#Borramos las que tengan sufijo  _y
eph.drop(eph.filter(regex='_y$').columns, axis=1, inplace=True)

print(len(eph))

eph.sample(10)


CODUSU        object
ANO4           int64
TRIMESTRE      int64
NRO_HOGAR      int64
COMPONENTE     int64
               ...  
VII1_2         int64
VII2_1         int64
VII2_2         int64
VII2_3         int64
VII2_4         int64
Length: 263, dtype: object
6706


CODUSU  ANO4  TRIMESTRE  NRO_HOGAR  COMPONENTE  \
366   TQRMNORPXHMMLSCDEIJAH00780187  2022          1          1           3   
433   TQRMNOQQQHLNTPCDEIJAH00780165  2022          1          1           1   
1154  TQRMNORUVHLNMQCDEIJAH00719271  2022          1          1           3   
3171  TQSMNOQUSHJMMPCDEIJAH00719547  2022          1          1           2   
6093  TQRMNORTXHMLRMCDEIJAH00698662  2022          1          1           2   
4165  TQRMNORTQHLMKRCDEIJAH00718914  2022          1          1           1   
2569  TQRMNOPSXHMNPNCDEIJAH00698516  2022          1          1           5   
1200  TQRMNOQQWHMMMUCDEIJAH00698722  2022          1          1           1   
5003  TQRMNOPPUHJMNRCDEIJAH00718707  2022          1          1           1   
3769  TQRMNOPUTHLMSUCDEIJAH00718897  2022          1          1           1   

      H15  REGION MAS_500  AGLOMERADO  PONDERA  ...  V19_B  IX_TOT IX_MEN10  \
366     0       1       S          33     1112  ...      2       3        1   
433     1       1       S          33      329  ...      2       3        0   
1154    0       1       S          33     3556  ...      2       6        3   
3171    1       1       S          33      941  ...      2       3        1   
6093    1       1       S          33     1698  ...      2       3        0   
4165    1       1       S          33     3062  ...      2       1        0   
2569    1       1       S          33     1565  ...      2       6        1   
1200    1       1       S          33     1209  ...      2       1        0   
5003    1       1       S          33     1497  ...      2       3        0   
3769    1       1       S          33     1522  ...      2       2        0   

      IX_MAYEQ10  VII1_1  VII1_2  VII2_1  VII2_2  VII2_3  VII2_4  
366            2       2       0       1       0       0       0  
433            3       3       1       2       0       0       0  
1154           3       1       0      98       0       0       0  
3171           2       2       0       1       0       0       0  
6093           3       2       1       3       0       0       0  
4165           1       1       0      98       0       0       0  
2569           5       1       0      98       0       0       0  
1200           1       1       0      98       0       0       0  
5003           3       1       0      98       0       0       0  
3769           2       1       2      98       0       0       0  

[10 rows x 242 columns]

4) 
- Para eliminar aquellas observaciones relacionadas a ingreso que sea negativo, utilizaremos el metodo .loc, combinando con el prefijo ~ para expresar que queremos localizar aquellas que no sean negativas en algun punto.
- Eliminamos duplicados con .drop_duplicates
- Con .isnull, .dropna y .shape eliminamos aquellas columnas que tengan mas de un porcentaje de NAs.
- Armando un interquantile range score eliminamos outliers.
- Con .dtypes nos fijamos el tipo de las variables restantes. 

5) 

In [21]:
#Eliminamos aquellas que tengan ingreso o edad negativa
eph = eph.loc[~(
(eph['CH06']<0) | 
(eph['PP08D1']<0) | 
(eph['PP08D4']<0) |
(eph['PP08F1']<0) |
(eph['PP08F2']<0) |
(eph['PP08J1']<0) |
(eph['PP08J2']<0) |
(eph['PP08J3']<0) |
(eph['ITF']<0) |
(eph['IPCF']<0) )]

In [22]:
#Chequeamos si hay duplicados y los dropeamos
eph = eph.drop_duplicates()
#no hay duplicados, cool!

In [23]:
#Cheque columnas con muchos NAs, si tiene mas de 70% los vamos a  borrar 
#Notese quiebre de NAs en 96% y 81%, no es tan arbitrario el 70%
percent_missing = eph.isnull().sum() * 100 / len(eph)
missing_value_eph = pd.DataFrame({'Columna': eph.columns,
                                 'Porcentaje de NAs': percent_missing})
missing_value_eph.sort_values('Porcentaje de NAs', inplace=True,ascending=False)
missing_value_eph.head(50)

,Columna,Porcentaje de NAs
PP09C_ESP,PP09C_ESP,100.000000
PDECCFR,PDECCFR,100.000000
PDECIFR,PDECIFR,100.000000
IDECIFR,IDECIFR,100.000000
PDECINDR,PDECINDR,100.000000
IDECINDR,IDECINDR,100.000000
PDECOCUR,PDECOCUR,100.000000
IDECOCUR,IDECOCUR,100.000000
IDECCFR,IDECCFR,100.000000
IV1_ESP,IV1_ESP,99.983311


In [24]:
#Borramos mas de 80%
perc = 70.0
min_count =  int(((100-perc)/100)*eph.shape[0] + 1)
eph = eph.dropna( axis=1, 
                thresh=min_count)
#chequeamos haber borrado lo que queriamos
percent_missing = eph.isnull().sum() * 100 / len(eph)
missing_value_eph = pd.DataFrame({'Columna': eph.columns,
                                 'Porcentaje de NAs': percent_missing})
missing_value_eph.sort_values('Porcentaje de NAs', inplace=True,ascending=False)
missing_value_eph.head(70)

,Columna,Porcentaje de NAs
PP09B,PP09B,61.214953
PP05C_2,PP05C_2,61.214953
PP07C,PP07C,61.214953
PP07A,PP07A,61.214953
PP06H,PP06H,61.214953
...,...,...
IV12_2,IV12_2,0.000000
IV12_3,IV12_3,0.000000
II1,II1,0.000000
II2,II2,0.000000


In [25]:
#Outliers: Nos quedaremos con los valores de ingreso entre los cuantiles
#siguiendo el metodo de interquantile range score propuesto en https://www.pluralsight.com/guides/cleaning-up-data-from-outliers

variables_para_outliers=[
'PP08D1','PP08D4','PP08F1','PP08F2','PP08J1','PP08J2','PP08J3','ITF','IPCF',
'P21', 'TOT_P12', 'P47T'
]

eph[variables_para_outliers].describe()


,PP08D1,PP08D4,PP08F1,PP08F2,PP08J1,PP08J2,PP08J3,ITF,IPCF,P21,TOT_P12,P47T
count,2324.000000,2324.0,2324.000000,2324.000000,2324.000000,2324.000000,2324.000000,5992.000000,5992.000000,5992.000000,5992.000000,5964.000000
mean,44783.868330,0.0,447.805508,88.339071,4674.591222,351.721170,21.944923,80452.365988,28244.142124,22295.271362,1560.551068,33829.937626
std,57861.612292,0.0,4645.968948,1517.465259,17444.385797,6560.869767,851.394948,106119.672606,42436.255827,48318.657494,10682.821820,59126.040833
min,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-9.000000,-9.000000,-9.000000
25%,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,30000.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,54500.000000,15750.000000,0.000000,0.000000,0.000000
75%,70000.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,120000.000000,40062.500000,30000.000000,0.000000,50000.000000
max,600000.000000,0.0,70000.000000,60000.000000,300000.000000,260000.000000,40000.000000,950000.000000,600000.000000,1000000.000000,240000.000000,910000.000000


In [26]:
#Generamos Interquantile Ranges, con eso sacamos los que queden afuera
#Asimetrico por ser ingreso una variable con lower bound pero no upper bound
Qlow = eph[variables_para_outliers].quantile(0.025)
Qhigh = eph[variables_para_outliers].quantile(0.90)
IQR = Qhigh - Qlow
print(IQR)
eph_inc = eph[variables_para_outliers]
eph_out = eph_inc[~((eph_inc < (Qlow - 1.5 * IQR)) |(eph_inc > (Qhigh + 1.5 * IQR))).any(axis=1)]
indexes = eph_out.index.values.tolist() 
eph.loc[indexes]

PP08D1     110000.0
PP08D4          0.0
PP08F1          0.0
PP08F2          0.0
PP08J1      13000.0
PP08J2          0.0
PP08J3          0.0
ITF        203000.0
IPCF        70800.0
P21         80009.0
TOT_P12         0.0
P47T        95709.0
dtype: float64


,CODUSU,ANO4,TRIMESTRE,NRO_HOGAR,COMPONENTE,H15,REGION,MAS_500,AGLOMERADO,PONDERA,...,V19_B,IX_TOT,IX_MEN10,IX_MAYEQ10,VII1_1,VII1_2,VII2_1,VII2_2,VII2_3,VII2_4
0,TQRMNOSUPHKKPQCDEIJAH00780151,2022,1,1,1,1,1,S,33,1741,...,2,3,0,3,1,2,3,0,0,0
1,TQRMNOSUPHKKPQCDEIJAH00780151,2022,1,1,2,1,1,S,33,1741,...,2,3,0,3,1,2,3,0,0,0
3,TQRMNOPQQHKMRLCDEIJAH00780169,2022,1,1,1,1,1,S,33,456,...,2,1,0,1,1,0,98,0,0,0
4,TQRMNOSXRHJMTRCDEIJAH00693084,2022,1,1,1,1,1,S,33,1199,...,2,2,0,2,1,3,98,0,0,0
5,TQRMNOSXRHJMTRCDEIJAH00693084,2022,1,1,3,1,1,S,33,1199,...,2,2,0,2,1,3,98,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6696,TQRMNOTRWHLMLUCDEIIAD00718588,2022,1,1,2,1,1,S,32,2554,...,2,2,0,2,96,0,1,0,0,0
6698,TQRMNOTSRHLMLUCDEIIAD00718231,2022,1,1,2,1,1,S,32,1327,...,2,2,0,2,1,0,2,0,0,0
6700,TQRMNOTTSHKNLSCDEIIAD00780102,2022,1,1,1,1,1,S,32,2839,...,2,2,0,2,1,0,98,0,0,0
6704,TQRMNOSRWHKMLUCDEIIAD00780103,2022,1,1,1,1,1,S,32,1513,...,2,2,0,2,1,0,2,0,0,0


In [27]:
#Chequeamos tipo de las variable restantes
eph.info(verbose=True)
print("Son todas object, int64 o float64, visto en una tabla:")
eph.dtypes.value_counts()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5992 entries, 0 to 6705
Data columns (total 198 columns):
 #    Column      Dtype  
---   ------      -----  
 0    CODUSU      object 
 1    ANO4        int64  
 2    TRIMESTRE   int64  
 3    NRO_HOGAR   int64  
 4    COMPONENTE  int64  
 5    H15         int64  
 6    REGION      int64  
 7    MAS_500     object 
 8    AGLOMERADO  int64  
 9    PONDERA     int64  
 10   CH03        int64  
 11   CH04        int64  
 12   CH05        object 
 13   CH06        int64  
 14   CH07        int64  
 15   CH08        int64  
 16   CH09        int64  
 17   CH10        int64  
 18   CH11        int64  
 19   CH12        int64  
 20   CH13        int64  
 21   CH14        float64
 22   CH15        int64  
 23   CH16        int64  
 24   NIVEL_ED    int64  
 25   ESTADO      int64  
 26   CAT_OCUP    int64  
 27   CAT_INAC    int64  
 28   PP02C1      int64  
 29   PP02C2      int64  
 30   PP02C3      int64  
 31   PP02C4      int64  
 32   PP

int64      126
float64     69
object       3
dtype: int64

In [29]:
#Definimos que variables no son categoricas
no_categoricas = ['CODUSU', 'CH06', 'MAS_500','ANO4','IDECOCUR',   
'IDECIFR','PP09C_ESP','CH16_COD','PP11D_COD','PP11C','PP11C99','PP11B2_DIA','PP11B2_MES','PP11B2_ANO','PP11T','PP11B_COD','PP11A','PP09A_ESP',
'IDECCFR','PDECOCUR','PDECCFR','IDECINDR','II8_ESP','PDECINDR','II7_ESP',    
'IV7_ESP','IV3_ESP','IV1_ESP','PDECIFR','IMPUTA','PP10E','PP10D','PP10C','PP10A','PP11B1','PP11G_ANO','PP11N','PP11O',
'PP11M','PP11L1','PP11L','PP11G_DIA', 'PP11G_MES','PP11S','PP11R','PP11Q','PP11P',  
'CH15_COD','PP03D', 'PP3E_TOT', 'PP3F_TOT', 'PP04B2', 'PP06C', 'PP06D', 'PP08D1', 'PP08D4', 'PP08F1', 'PP08F2', 'PP08J1', 
'PP08J2', 'PP08J3', 'P21', 'DECOCUR','RDECOCUR', 'GDECOCUR', 'ADECOCUR', 'PONDIIO', 'TOT_P12', 
'P47T', 'DECINDR', 'RDECINDR', 'GDECINDR', 'ADECINDR', 'PONDII', 'V2_M', 'V3_M', 'V4_M', 'V5_M', 
'V8_M', 'V9_M', 'V10_M', 'V11_M', 'V12_M', 'V18_M', 'V19_AM', 'V21_M', 'T_VI', 'ITF', 'DECIFR', 'RDECIFR', 'GDECIFR', 
'ADECIFR','IPCF', 'DECCFR', 'RDECCFR', 'GDECCFR', 'ADECCFR', 'PONDIH','REALIZADA',  'NRO_HOGAR', 'COMPONENTE', 'H15']
#Me fijo cuales fueron borradas
no_borradas = pd.Index(no_categoricas).difference(eph.columns).tolist()
#las saco
no_categoricas = 


['CH15_COD', 'CH16_COD', 'IDECCFR', 'IDECIFR', 'IDECINDR', 'IDECOCUR', 'II7_ESP', 'II8_ESP', 'IMPUTA', 'IV1_ESP', 'IV3_ESP', 'IV7_ESP', 'PDECCFR', 'PDECIFR', 'PDECINDR', 'PDECOCUR', 'PP09A_ESP', 'PP09C_ESP', 'PP10A', 'PP10C', 'PP10D', 'PP10E', 'PP11A', 'PP11B1', 'PP11B2_ANO', 'PP11B2_DIA', 'PP11B2_MES', 'PP11B_COD', 'PP11C', 'PP11C99', 'PP11D_COD', 'PP11G_ANO', 'PP11G_DIA', 'PP11G_MES', 'PP11L', 'PP11L1', 'PP11M', 'PP11N', 'PP11O', 'PP11P', 'PP11Q', 'PP11R', 'PP11S', 'PP11T']


6) 

7. Agregamos la columna adulto_equiv y adulto_equiv_hogar

In [13]:
df_equiv = pd.read_excel('tabla_adulto_equiv.xlsx')

df_equiv.loc[1,"Edad"]=1


for i in range(0,18):
    df_equiv.loc[i,"Edad"]= i 

for i in range(18,30): #18-29 años 0.76/1.02
    df_equiv.loc[i]= [i,0.76,1.02] 

for i in range(30,46): #30-45 años 0.77/1
    df_equiv.loc[i]= [i,0.77,1] 

for i in range(46,61): #46-60 años 0.76/1
    df_equiv.loc[i]= [i,0.76,1] 

for i in range(61,76): #61-75 años 0.67/0.83
    df_equiv.loc[i]= [i,0.67,0.83]

for i in range(76,106): #18-29 años 0.63/0.74. La edad max en la EPH es 105 años. 
    df_equiv.loc[i]= [i,0.63,0.74] 

df_equiv = df_equiv.rename(columns={'Edad':'CH06'})
print(df_equiv)

      CH06  Mujeres  Varones
0        0     0.35     0.35
1        1     0.37     0.37
2        2     0.46     0.46
3        3     0.51     0.51
4        4     0.55     0.55
..     ...      ...      ...
101  101.0     0.63     0.74
102  102.0     0.63     0.74
103  103.0     0.63     0.74
104  104.0     0.63     0.74
105  105.0     0.63     0.74

[106 rows x 3 columns]


In [14]:
df_eph= eph.merge(df_equiv, on='CH06')

for i in range(len(df_eph)):
    if df_eph.loc[i,"CH04"]==1:
        df_eph.loc[i,"adulto_equiv"]=df_eph.loc[i,"Varones"]
    elif df_eph.loc[i,"CH04"]==2:
        df_eph.loc[i,"adulto_equiv"]=df_eph.loc[i,"Mujeres"]


df_eph.drop (["Varones","Mujeres"] , axis =1 , inplace = True)

In [15]:
df_sum=df_eph.groupby(['CODUSU', 'NRO_HOGAR'])[['adulto_equiv']].agg('sum')
df_sum.reset_index( inplace = True )
df_sum = df_sum.rename(columns={'adulto_equiv':'ad_equiv_hogar'})

df_eph=df_eph.merge(df_sum, on=["CODUSU", "NRO_HOGAR"])
df_eph.sort_values(by="CODUSU")

#El data frame relevante ahora es df_eph


,CODUSU,ANO4,TRIMESTRE,NRO_HOGAR,COMPONENTE,H15,REGION,MAS_500,AGLOMERADO,PONDERA,...,IX_MEN10,IX_MAYEQ10,VII1_1,VII1_2,VII2_1,VII2_2,VII2_3,VII2_4,adulto_equiv,ad_equiv_hogar
4273,TQRMNOPPQHJMNRCDEIJAH00693035,2022,1,1,1,1,1,S,33,1427,...,3,2,2,0,98,0,0,0,1.02,2.88
4275,TQRMNOPPQHJMNRCDEIJAH00693035,2022,1,1,3,0,1,S,33,1427,...,3,2,2,0,98,0,0,0,0.64,2.88
4276,TQRMNOPPQHJMNRCDEIJAH00693035,2022,1,1,4,0,1,S,33,1427,...,3,2,2,0,98,0,0,0,0.46,2.88
4274,TQRMNOPPQHJMNRCDEIJAH00693035,2022,1,1,2,1,1,S,33,1427,...,3,2,2,0,98,0,0,0,0.76,2.88
5368,TQRMNOPPQHJMQLCDEIJAH00702455,2022,1,1,1,1,1,S,33,2054,...,0,2,1,2,98,0,0,0,0.67,1.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4235,TQUMNORTQHLMNUCDEIJAH00780261,2022,1,1,6,0,1,S,33,2031,...,1,6,1,7,2,0,0,0,0.60,5.64
4236,TQUMNORTQHLMNUCDEIJAH00780261,2022,1,1,2,1,1,S,33,2031,...,1,6,1,7,2,0,0,0,1.02,5.64
4237,TQUMNORTQHLMNUCDEIJAH00780261,2022,1,1,1,1,1,S,33,2031,...,1,6,1,7,2,0,0,0,0.77,5.64
4238,TQUMNORTQHLMNUCDEIJAH00780261,2022,1,1,5,1,1,S,33,2031,...,1,6,1,7,2,0,0,0,0.82,5.64


8. Dividimos la base en respondieron y no respondieron 

In [16]:
#Guardamos las obs que respondieron en el df 'df_respondieron':
df_respondieron = df_eph.drop(df_eph[df_eph['ITF']<=0].index)
df_respondieron.reset_index(inplace=True, drop=True)

##Guardamos las obs que no respondieron en el df 'df_norespondieron':
df_norespondieron= df_eph.drop(df_eph[df_eph['ITF']>0].index) 
df_norespondieron.reset_index( inplace = True )
print(len(df_norespondieron)) #chequeamos que la cantidad de obs sea igual a no_resp

2250


Agregamos la variable "ingreso_necesario"

In [17]:
df_respondieron["ingreso_necesario"]=df_respondieron["ad_equiv_hogar"]*(27197.64) #$27197.64 es la CBA

9. Agregamos la variable "pobre"

In [18]:
df_respondieron["Pobre"]=0

for i in range(len(df_respondieron)):
    if df_respondieron.loc[i,"ingreso_necesario"] < 27197.64 :
        df_respondieron.loc[i,"Pobre"]= 1

10. Calculamos la tasa de hogares pobres
como se usa el PONDIH????

In [ ]:
df_respondieron.sort_values(by="CODUSU")

pobres=[]

for i in range(len(df_respondieron)):
    if df_respondieron.loc[i,"Pobre"] == "1" and df_respondieron.loc[i,"CODUSU"]!= df_respondieron.loc[i-1,"CODUSU"] :
        pobres.apend( df_respondieron.loc[i,"Pobre"]* df_respondieron.loc[i,"PONDICH"] )


len(df_eph['CODUSU'].unique()) #hogares


2327

## PARTE II

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import mean_squared_error

In [ ]:
def evalua_metodo(X_train, X_test, y_train, y_test, model):
   '''    
   Toma como parametros las bases de train y test y el modelo deseado: LogisticRegression, lda, knn.
   Devuelve las métricas: Matriz de confusión, AUC, ROC y accuracy 
   '''

   y = model.fit(X_train, y_train)
   y_pred = y.predict(X_test)

   matriz_confusion = confusion_matrix(y_test, y_pred)
   auc = roc_auc_score(y_test, y_pred)
   fpr, tpr, tresholds = roc_curve(y_test, y_pred)
   accuracy = accuracy_score(y_test, y_pred)
   ecm = mean_squared_error(y_pred, y_test)

   df_metrics = pd.DataFrame({'modelo': [model], 'accuracy': [accuracy], 'auc':[auc], 'ecm':[ecm]})
   
   return df_metrics


In [ ]:
from sklearn.model_selection import KFold
import statistics

In [ ]:
def cross_validation(model, k, x, y):
    kf = KFold(n_splits=k, shuffle=True)

    ecm = []

    for i, (train_index, test_index) in enumerate(kf.split(x)):   
        x_train, x_test = x.iloc[list(train_index)], x.iloc[list(test_index)]  
        y_train, y_test = y.iloc[list(train_index)], y.iloc[list(test_index)]

        ecm2 = evalua_metodo(x_train, x_test, y_train, y_test, model)
        ecm3 = ecm2.iloc[0]['ecm']
        ecm.append(ecm3)

    final_ecm = statistics.mean(ecm)

    return final_ecm
    